<a href="https://colab.research.google.com/github/harrisonxia/Predictive-Maintenance/blob/master/iot_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
uploaded = files.upload()

Saving part-00000-9bbc2c66-17ba-4260-964c-d4f40fd03234-c000.csv to part-00000-9bbc2c66-17ba-4260-964c-d4f40fd03234-c000.csv


In [0]:
import io
import math
import pandas as pd
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
from tensorflow import placeholder, float32, reshape, Variable, random_normal, matmul, global_variables_initializer
from tensorflow import device, Session, reset_default_graph, reduce_sum, square
from tensorflow.train import AdamOptimizer
from tensorflow.nn import tanh, dynamic_rnn
from tensorflow.contrib.rnn import LSTMCell, MultiRNNCell
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import SelectKBest, chi2

def loadCSV():
  df = pd.read_csv(io.BytesIO(uploaded['part-00000-9bbc2c66-17ba-4260-964c-d4f40fd03234-c000.csv'])).sort_values(
	 ['TimeStamp'], ascending = True).reset_index()

  df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])
  
  df.drop(['::[scararobot]Ax_J1.PositionCommand','::[scararobot]Ax_J1.TorqueFeedback','::[scararobot]Ax_J2.PositionCommand','::[scararobot]Ax_J2.TorqueFeedback','::[scararobot]Ax_J3.TorqueFeedback','::[scararobot]Ax_J6.TorqueFeedback','::[scararobot]ScanTimeAverage','::[scararobot]Ax_J6.PositionCommand','::[scararobot]Ax_J3.PositionCommand','index'], axis=1, inplace=True)
  df['Total']= df.select_dtypes(include=['float64','float32']).apply(lambda row: np.sum(row),axis=1)
  return pd.Series(df['Total'])

def featureSelect(df):
  x = df.iloc[0:, 1: 20].values
  y = df.iloc[0:, 20: 21].values.flatten()
  y = np.array(y).astype(int)
  
  SelBest = SelectKBest(chi2, k = 10)
  SelBest.fit_transform(x, y)
  choosed_features = list(SelBest.get_support())
  f_df = df.iloc[0:, 0: 1]
  for index, fea in enumerate(choosed_features):
    if fea == True:
      f_df[str(index)] = df.iloc[0:, index]
  
  f_df['Total'] = f_df.select_dtypes(include = ['float64', 'float32']).apply(lambda row: np.sum(row), axis = 1)
  tensor = pd.Series(f_df['Total'])
  return tensor

def prepareData(num_periods, f_horizon, TS, test_size = 1):
  #create our training input data set "X"
  x_data = TS[:(len(TS)-(len(TS) % num_periods))]
  x_batches = x_data.reshape(-1, num_periods, 1)

  #create our training output dataset "y"
  y_data = TS[1:(len(TS)-(len(TS) % num_periods))+f_horizon]
  print (y_data)
  print (len(y_data))
  y_batches = y_data.reshape(-1, num_periods, 1)
  print (len(y_batches))
  
  #create our test X and y data
  test_x_setup = TS[-(num_periods * test_size + f_horizon):]
  testX = test_x_setup[:num_periods * test_size].reshape(-1, num_periods, 1)
  testY = TS[-(num_periods * test_size):].reshape(-1, num_periods, 1)
  print (testX.shape)
  print (testX[:,(num_periods-1):num_periods])
  print (testY.shape)
  print (testY[:,(num_periods-1):num_periods])
  return x_batches, y_batches, testX, testY

def RNN_LSTM(num_periods, input_size, hidden_number, output_size):
  
  X = placeholder(float32, [None, num_periods, input_size], name = "X")   #create variable objects
  Y = placeholder(float32, [None, num_periods, output_size], name = "Y")

  LSTM_cell_1 = LSTMCell(num_units = hidden_number, activation = tanh)
  LSTM_cell_2 = LSTMCell(num_units = 100)
  stacked_lstm_cell = MultiRNNCell([LSTM_cell_1, LSTM_cell_2])
  lstm_output, states = dynamic_rnn(stacked_lstm_cell, X, dtype=float32)               #choose dynamic over static
  
  stacked_rnn_output = reshape(lstm_output, [-1, hidden_number])           #change the form into a tensor
  weight = Variable(random_normal([100, 1]))   
  bias = Variable(random_normal([1]))   
  stacked_outputs = matmul(stacked_rnn_output, weight) + bias  
  f_outputs = reshape(stacked_outputs, [-1, num_periods, output_size])          #shape of results
  return f_outputs, X, Y

def Train(lr, train_op, cost, epochs, dev, X, Y, x_batches, y_batches, X_test, f_outputs):
  init_g = global_variables_initializer()
  loss = []
  with device(dev):
    with Session() as session:
      session.run(init_g)
      for ep in range(epochs):
        session.run(train_op, feed_dict = {X: x_batches, Y: y_batches})
        if ep % 10 == 0:
          mse = cost.eval(session = session, feed_dict = {X: x_batches, Y: y_batches})
          mse = math.sqrt(mse)
          if ep % 100 == 0:
            print(ep, "\tMSE:", mse)
          loss.append(mse)
      y_pred = session.run(f_outputs, feed_dict = {X: X_test})
  return session, y_pred, loss

#Plot our test y data and our y-predicted forecast
def plotPred(y_pred, Y_test):
  plt.title("Forecast vs Actual", fontsize=14)
  plt.plot(pd.Series(np.ravel(Y_test)), "bo", markersize=10, label="Actual")
  plt.plot(pd.Series(np.ravel(y_pred)), "r.", markersize=10, label="Forecast")
  plt.legend(loc="upper right")
  plt.xlabel("Time Periods")
  plt.show()
  return

def plotLoss(loss):
  plt.title("Training Loss", fontsize=14)
  plt.plot(loss)
  plt.xlabel("Time Periods")
  plt.show()
  return

#anomaly detection
def anomalyDetection(trainset, pred, outliers_fraction, model = 'SVM'):
  if model == 'SVM':
    m = OneClassSVM(nu=0.95 * outliers_fraction, kernel="rbf",gamma='scale')
  elif model == 'IF':
    m = IsolationForest(contamination = outliers_fraction)
  m.fit(trainset)
  column = 'anomaly_' + model
  pred[column] = pd.Series(m.predict(pred))
  result = pred.loc[pred[column] == -1, ['Prediction']]
  pred = pred.drop(columns = [column])
  return result, pred

def plotAD(pred, SVM, IF):
  fig, ax = plt.subplots(figsize=(15,9))

  ax.plot(pred['Prediction'], color='blue', label = 'Prediction')
  ax.scatter(list(SVM.index.values), list(SVM.Prediction.values), color = 'red', label = 'anomalySVM')
  ax.scatter(list(IF.index.values), list(IF.Prediction.values), color = 'green', label = 'anomalyIF')
  plt.legend(loc = 'upper right')
  plt.show()
  return

if __name__ == '__main__':
  #tensor = featureSelect(loadCSV())
  tensor = loadCSV()
  num_periods = 100
  f_horizon = 1       #number of periods into the future we are forecasting
  TS = np.array(tensor)   #convert time series object to an array
  
  x_batches, y_batches, X_test, Y_test = prepareData(num_periods, f_horizon, TS)
  
  #set up model parameters
  reset_default_graph()   #reset graph
  input_size = 1            #number of vectors submitted
  hidden_number = 100          #number of neurons
  output_size = 1            #number of output vectors

  f_outputs, X, Y = RNN_LSTM(num_periods, input_size, hidden_number, output_size)
  
  #set up training parameters
  learning_rate = 0.005   #learning rate
  cost = reduce_sum(square(f_outputs - Y), name = 'cost')    #define the cost function
  optimizer = AdamOptimizer(learning_rate = learning_rate)          #gradient descent method
  train_op = optimizer.minimize(cost)          #train the result of the application of the cost_function                                 
  epochs = 1000     #number of iterations or training cycles, includes both the FeedFoward and Backpropogation
  
  session, y_pred, loss = Train(learning_rate, train_op, cost, epochs, '/device:GPU:0', X, Y, x_batches, y_batches, X_test, f_outputs)

  DIR="/usr/tmp/IoTModel"  #path where the model will be saved
  #tf.saved_model.simple_save(session, DIR,
  #                         inputs={"X": X},
  #                         outputs={"outputs": f_outputs})
  
  plotPred(y_pred, Y_test)
  plotLoss(loss)
  
  #prepare for anomaly detection
  pred = pd.DataFrame(np.ravel(y_pred), columns = ['Prediction'])
  scaler = StandardScaler()
  np_scaled = scaler.fit_transform(pred)
  pred = pd.DataFrame(np_scaled, columns = ['Prediction'])
  
  testDF = pd.DataFrame(TS, columns = ['Total'])
  testDF_scaled = scaler.fit_transform(testDF)
  testDF = pd.DataFrame(testDF_scaled, columns = ['Total'])
  
  SVM, pred = anomalyDetection(testDF, pred, 0.002)
  IF, pred = anomalyDetection(testDF, pred, 0.01, 'IF')
  plotAD(pred, SVM, IF)

  
  
  



[ 327.414406    0.        327.791445 ...    0.       -128.657941
    0.      ]
81900
819
(1, 100, 1)
[[[0.]]]
(1, 100, 1)
[[[4.029848]]]
0 	MSE: 42929.79496806385
100 	MSE: 25576.38410721891
200 	MSE: 19626.49556084835
300 	MSE: 15275.65697441521
400 	MSE: 12285.643654282017
500 	MSE: 10144.410874959669


In [0]:
pred

,Prediction,anomaly_SVM
0,0.610533,1
1,3.439323,-1
2,0.351026,1
3,2.369972,-1
4,0.351380,1
5,1.655158,1
6,0.342733,1
7,1.455347,1
8,0.339639,1
9,1.519618,1
